In [1]:
import pickle
import os, random
import h5py, copy
import numpy as np
import pandas as pd
from tqdm import tqdm

### Torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset

### Torchvision
from torchsummary import summary
import torchvision.datasets as dest
import torchvision.transforms as transformers

### Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
### seed_everythin
seed = 1987
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.random.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Parameters

In [4]:
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(f'Device: {device}')

Device: cuda:0


### Read data from pickle

In [5]:
filename = 'selected_data.pkl'
outfile = open(filename,'rb')
data = pickle.load(outfile)
xdata = np.array(data['data'], dtype = 'float32')
ylablel = np.array(data['labels'], dtype = 'float32')

### Train test split

In [6]:
### Split data
X_train, X_val, y_train, y_val = train_test_split(xdata, ylablel, test_size=0.2)
X_train.shape, X_val.shape

((145864, 50, 3), (36466, 50, 3))

### Normalize data

In [7]:
xscalers = {}
for i in range(xdata.shape[2]):
    xscalers[i] = StandardScaler()
    X_train[:, :, i] = xscalers[i].fit_transform(X_train[:, :, i])

yscaler = StandardScaler()
y_train = yscaler.fit_transform(y_train.reshape(-1, 1))

In [8]:
scalers = {}
for i in range(xdata.shape[2]):
    X_val[:, :, i] = xscalers[i].transform(X_val[:, :, i])

y_val = yscaler.transform(y_val.reshape(-1, 1))

In [9]:
y_val.max(), y_train.max()

(6.706501, 7.02627)

### Create dataloader

In [28]:
### Dataset
class SeismicDataset(Dataset):
    def __init__(self, method):
        if method == 'train':
            self.xdata = torch.Tensor(X_train)
            self.ylablel = torch.Tensor(y_train)
        elif method == 'val':
            self.xdata = torch.Tensor(X_val)
            self.ylablel = torch.Tensor(y_val)
            
        
    def __len__(self):
        return len(self.xdata)
        
    def __getitem__(self, indx):
        return self.xdata[indx], self.ylablel[indx]
        
### Dataloader
batch_size = 512
num_workers = 1

train_dataloader = torch.utils.data.DataLoader(
    SeismicDataset('train'),
    batch_size= batch_size,
    shuffle = True,
    num_workers = num_workers
)

val_dataloader = torch.utils.data.DataLoader(
    SeismicDataset('val'),
    batch_size= batch_size,
    shuffle = True,
    num_workers = num_workers
)

### Model

In [29]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(ConvBlock, self).__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv1d(in_channels, out_channels,  bias = False, **kwargs)
        self.batchnorm = nn.BatchNorm1d(out_channels)
        
    def forward(self, x):
        return self.relu(self.batchnorm(self.conv(x)))

In [30]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1_pool):
        super(InceptionBlock, self).__init__()
        self.branch1 = ConvBlock(in_channels, out_1x1, kernel_size = 1)
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, red_3x3, kernel_size = 1),
            ConvBlock(red_3x3, out_3x3, kernel_size = 3, padding = 1)
        )
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, red_5x5, kernel_size = 1),
            ConvBlock(red_5x5, out_5x5, kernel_size = 5, padding = 2)
        )
        
        self.branch4 = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride = 1, padding = 1),
            ConvBlock(in_channels, out_1x1_pool, kernel_size = 1)
        )
        
        
    def forward(self, x):
        return torch.cat(
            [self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1
        )
       

In [31]:
class SeismicNet(nn.Module):
    def __init__(self):
        super(SeismicNet, self).__init__()
        self.conv1 = ConvBlock(3, 192, kernel_size = 2, stride = 2)
        self.inception_1a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception_1b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.maxpool1 = nn.MaxPool1d(kernel_size=3, stride=2, padding = 1)
        
        self.averagepool1 = nn.AvgPool1d(kernel_size= 7, stride= 1)
        self.fc1 = nn.Linear(3360, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 1)
        self.dropout1 = nn.Dropout2d(p = 0.25)
        self.dropout2 = nn.Dropout2d(p = 0.20)
        self.dropout3 = nn.Dropout2d(p = 0.15)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.inception_1a(x)
        x = self.inception_1b(x)
        x = self.maxpool1(x)

        x = self.averagepool1(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.dropout3(x)
        x = self.fc3(x)
        return x

In [32]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(150, 128),
            nn.ReLU(),
            nn.Dropout(0.35),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.35),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
        
    def forward(self, x):
        x = Variable(torch.flatten(x, start_dim = 1))
        x = self.main(x)
        return x

In [33]:
model = SimpleModel().to(device) #SeismicNet().to(device)
if (device.type == 'cuda' and (ngpu> 1)):
    model = nn.DataParallel(model, list(range(ngpu)))

In [34]:
# summary(model, input_size=(3, 50))
# print(model)

### Training Model

In [35]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=10, factor = 0.15, verbose=True
)

In [36]:
def train():
    model.train()
    losses = []
    for j, (data, label) in enumerate(train_dataloader, 0):
        optimizer.zero_grad()
        data = data.to(device).view(data.shape[0], data.shape[2], data.shape[1])
        y = label.to(device)
        output = model(data)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    mean_loss = sum(losses)/len(losses)
    return mean_loss
        
def evaluate():
    model.eval()
    with torch.no_grad():
        losses = []
        for j, (data, label) in enumerate(val_dataloader, 0):
            data = data.to(device).view(data.shape[0], data.shape[2], data.shape[1])
            y = label.to(device)
            output = model(data)
            losses.append(criterion(output, y).item())
    return sum(losses)/len(losses)

In [37]:
num_epoch = 500
train_loss = []
val_loss = []
best_val_loss = np.inf

for i, epoch in enumerate(range(num_epoch)):
    train_loss.append(train())
    val_loss.append(evaluate())
    
    print(f'| Epoch: {epoch}/{num_epoch} | Train loss: {train_loss[i]} | Val loss: {val_loss[i]}')
        
    if val_loss[i] < best_val_loss:
        best_val_loss = val_loss[i]
        best_model = model
        
        best_model_weights = copy.deepcopy(model.state_dict())
        torch.save(best_model_weights, f'./best_models/seismic_net_epoch_{epoch}_val_loss_{val_loss[i]}.pt')

    scheduler.step(val_loss[i])
    

| Epoch: 0/500 | Train loss: 1.0436619532735725 | Val loss: 0.9891992029216554
| Epoch: 1/500 | Train loss: 0.9935212423926906 | Val loss: 0.971724627746476
| Epoch: 2/500 | Train loss: 0.9795651847856086 | Val loss: 0.9667215968171755
| Epoch: 3/500 | Train loss: 1.0038976186200192 | Val loss: 0.9596497606900003
| Epoch: 4/500 | Train loss: 1.8748206607082434 | Val loss: 0.9528772864076827
| Epoch: 5/500 | Train loss: 1.1947972155453865 | Val loss: 0.9452766593959596
| Epoch: 6/500 | Train loss: 0.984333312302305 | Val loss: 0.9357115195857154
| Epoch: 7/500 | Train loss: 0.9474844940921716 | Val loss: 0.9342307878865136
| Epoch: 8/500 | Train loss: 0.9675219717778658 | Val loss: 0.9270032155844901
| Epoch: 9/500 | Train loss: 0.9283860269345735 | Val loss: 0.9121393660704294
| Epoch: 10/500 | Train loss: 0.9347056719294765 | Val loss: 0.9076041819320785
| Epoch: 11/500 | Train loss: 0.9180914046471579 | Val loss: 0.900248609483242
| Epoch: 12/500 | Train loss: 0.9132980635291652 | Va

### Train and validation error

In [27]:
title = f'Minimum validation loss: {np.min(val_loss)}'
p = figure(title = title, plot_width=600, plot_height=250)
x = np.arange(len(train_loss))
p.line(x, train_loss, line_width=2, color = 'blue', legend_label = 'Train RMSE')
p.line(x, val_loss, line_width=2, color = 'red', legend_label = 'Val RMSE')
show(p)